In [0]:
# TODO BERT
# TODO GPT2
# TODO eli5
# TODO wandb
# TODO Прибраться в ноутбуке

In [0]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd

from transformers import DistilBertTokenizer, DistilBertModel, BertModel, GPT2Tokenizer, DistilBertForSequenceClassification

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext  import data

from tqdm import tqdm_notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data

In [0]:
def open_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        text_list = [line for line in f.readlines()]
    return text_list

In [4]:
# uncomment if google colab:
import os 
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

fake = open_file("data/fake.txt")
real = open_file("data/real.txt")
df = pd.read_csv("data/dataset.csv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
print(len(fake), len(real), df.shape)
print(fake[:2])
print(real[:2])
df.head()

37366 37366 (74732, 2)
['Spinach has terrorized generations of veggie-phobic kids, and many grownups don\'t much like it, either.."I think it\'s a little bit of a shock to see that he\'s been able to do this,"\n', 'All day, every day, Cheryl Bernstein thanks her 16-month-old son. the boy is a little boy.\n']
["Spinach has terrorized generations of veggie-phobic kids, and many grownups don't much like it, either. But when it's combined with seasonings and feta cheese and wrapped in a golden crisp phyllo dough crust, even those who despise Popeye's Â\xadfavorite food ask for seconds.\n", 'All day, every day, Cheryl Bernstein thanks her 16-month-old son. "I gave life to Reid, but he gave me life - a reason to get clean and go on," she said yesterday after graduating from the Manhattan Family Treatment Court program.\n']


,text,label
0,"Is a skull from Petralona Cave, Greece, the ol...",real
1,The Network Readiness Index published by the W...,fake
2,Now they've got Justin Bieber too. He was just...,real
3,"NOGALES, Arizona — Jessica Elizabeth Orellana ...",real
4,Many companies that are using cloud computing ...,fake


# BERT

In [6]:
# BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
#                       BertForSequenceClassification, BertForTokenClassification, BertForQuestionAnswering]
pretrained_weights = 'distilbert-base-cased'
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights)
model = DistilBertModel.from_pretrained(pretrained_weights)

In [7]:
embeddings_pretrained = model.get_input_embeddings()
embeddings_pretrained

Embedding(28996, 768, padding_idx=0)

In [8]:
def tokenize(text, tokenizer=tokenizer):
    return tokenizer.encode(text)

print(tokenize(fake[0]))
print(tokenize(real[0]))

[101, 22878, 7291, 1144, 8938, 2200, 8225, 1104, 1396, 9705, 1663, 118, 185, 5114, 15421, 4067, 117, 1105, 1242, 4215, 17210, 1274, 112, 189, 1277, 1176, 1122, 117, 1719, 119, 119, 107, 146, 1341, 1122, 112, 188, 170, 1376, 2113, 1104, 170, 4900, 1106, 1267, 1115, 1119, 112, 188, 1151, 1682, 1106, 1202, 1142, 117, 107, 102]
[101, 22878, 7291, 1144, 8938, 2200, 8225, 1104, 1396, 9705, 1663, 118, 185, 5114, 15421, 4067, 117, 1105, 1242, 4215, 17210, 1274, 112, 189, 1277, 1176, 1122, 117, 1719, 119, 1252, 1165, 1122, 112, 188, 3490, 1114, 1265, 5018, 1105, 175, 12405, 9553, 1105, 4293, 1107, 170, 5404, 19501, 185, 18873, 2858, 28075, 21508, 117, 1256, 1343, 1150, 3532, 19093, 1162, 4409, 4980, 112, 188, 228, 8057, 12198, 3150, 2094, 2367, 1111, 3071, 119, 102]


In [9]:
MAX_VOCAB_SIZE = 50000
classes={'fake': 0, 'real': 1}


TEXT = data.Field(sequential=True, include_lengths=False, batch_first=True, tokenize=tokenize, 
             pad_first=True, lower=False) 
LABEL = data.LabelField(dtype=torch.long, use_vocab=True, preprocessing=lambda x: classes[x])

dataset = data.TabularDataset('data/dataset.csv', 
                                format='csv', fields=[('text', TEXT), ('label',LABEL),], 
                                skip_header=True)

TEXT.build_vocab(dataset,  max_size=MAX_VOCAB_SIZE, min_freq=2)
LABEL.build_vocab(dataset)
vocab = TEXT.vocab
print('Vocab size:', len(TEXT.vocab.itos))

train, test = dataset.split(0.8, stratified=True)
train, valid = train.split(0.8, stratified=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (915 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors


Vocab size: 25023


In [22]:
batch_size = 4
num_epochs = 10

model = DistilBertForSequenceClassification.from_pretrained(pretrained_weights)
model.to(device)
# wandb.watch(model)
# wandb.init(project="dpl")

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    device=device,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)
#train_iterator = BucketIteratorWrapper(train_iterator)
#valid_iterator = BucketIteratorWrapper(valid_iterator)
#test_iterator = BucketIteratorWrapper(test_iterator)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True, cooldown=5)
criterion = nn.BCEWithLogitsLoss()
criterion.to(device)

BCEWithLogitsLoss()

In [25]:
model.distilbert.requires_grad

AttributeError: ignored

In [32]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad_)
print(pytorch_total_params)

model.distilbert.requires_grad_ = False
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad_)
print(pytorch_total_params)

65783042
65783042


In [0]:
model.distilbert.requires_grad_?

In [0]:
def accuracy_score(preds, y):
    # preds = torch.round(torch.sigmoid(preds))
    preds = (preds == y).float()
    accuracy = preds.sum() / len(preds)
    return accuracy.item()

In [0]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):
    model.train()

    running_loss = 0
    losses = []
    train_acc = []

    n_batches = len(iterator)
    
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)
    
    for i, batch in enumerate(iterator):
        x = batch.text
        y = batch.label
        optimizer.zero_grad()
        preds = model(x, labels=y)
        loss, logits = preds
        _, preds = torch.max(logits.data, 1)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        curr_loss = loss.data.detach().item()
        losses.append(loss.item())

        acc = accuracy_score(preds, y)
        train_acc.append(acc)

        iterator.set_postfix(loss='%.5f' % curr_loss, acc='%.5f' % acc)

        # wandb.log({
        # "Train Accuracy": np.mean(train_acc),
        # "Train Loss": curr_loss})

    return curr_loss, losses, train_acc

def _test_epoch(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    losses = []
    test_acc = []

    n_batches = len(iterator)

    with torch.no_grad():
        for batch in iterator:
            x = batch.text
            y = batch.label
            preds = model(x, labels=y)
            loss, logits = preds
            _, preds = torch.max(logits.data, 1)
            test_acc.append(accuracy_score(preds, y))

            loss = None
            losses.append(loss.item())
            epoch_loss += loss.data.item()

    # wandb.log({
    #     "Valid Accuracy": np.mean(test_acc),
    #     "Valid Loss": epoch_loss/n_batches})
    
    return epoch_loss / n_batches, losses, test_acc

def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, n_epochs=20, early_stopping=0):

    prev_loss = 10500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in range(n_epochs):
        train_loss, epoch_tl, train_acc = _train_epoch(model, train_iterator, optimizer, criterion, epoch)
        valid_loss,  epoch_vl, valid_acc = _test_epoch(model, valid_iterator, criterion)

        train_losses.extend(epoch_tl)
        valid_losses.extend(epoch_vl)
        train_accs.extend(train_acc)
        valid_accs.extend(valid_acc)

        print('validation loss %.5f' % valid_loss, 'validation accuracy  %.5f' % np.mean(valid_accs))

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss, 'train_acc': np.mean(train_accs), 'valid_acc':np.mean(valid_accs)}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)
    return history

In [0]:
history = nn_train(model, train_iterator, valid_iterator,
          criterion, optimizer, n_epochs=100, early_stopping=2)

RuntimeError: ignored

In [0]:
device

device(type='cuda')

In [0]:
def test_model(model, test_iterator):
    test_acc = []

    with torch.no_grad():
        for item in test_iterator:
            #x = item['features']
            #y = item['targets']
            x = item.text
            y = item.label
            preds = model(x).squeeze(1)
            test_acc.append(accuracy_score(preds, y))
    #print(type(test_acc), type(test_acc[0]))
    test_acc = np.mean(test_acc) 
    return np.mean(test_acc)

In [0]:
test_accuracy = test_model(model, test_iterator)
print('Test accuracy: {}'.format(np.mean(test_accuracy)))

RuntimeError: ignored

# ANOTHER BERT

In [0]:
pretrained_weights = 'distilbert-base-cased'
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights)
model = DistilBertModel.from_pretrained(pretrained_weights)

In [0]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (915 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors


In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(74732, 917)

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(74732, 917)

In [0]:
input_ids = torch.tensor(padded[:10000])  
attention_mask = torch.tensor(attention_mask[:10000])

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
labels = df['label'][:10000]

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [0]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)